In [18]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from clsfr_data_loading import lungCT_clsfr_DATA
import numpy as np

In [21]:
class lungPartitionClsfr(nn.Module):
    def __init__(self, in_c, out_c):
        super(lungPartitionClsfr, self).__init__()
        self.max_pool2d = nn.MaxPool2d(2)
        self.down_conv1 = self.double_conv(in_c, 64)
        self.down_conv2 = self.double_conv(64, 64)
        self.down_conv3 = self.double_conv(64, 128)
        self.down_conv4 = self.double_conv(128, 128)
        self.linear1 = nn.Linear(28*28*128, 512)
        self.linear2 = nn.Linear(512, 128)
        self.linear3 = nn.Linear(128, out_c)
        
    def double_conv(self, in_c, out_c, kernel_size = 3, padding = 0):
        conv = nn.Sequential(
            nn.Conv2d(in_c, out_c, kernel_size = kernel_size, padding = padding),
            nn.BatchNorm2d(out_c),
            nn.ReLU(inplace = True),
            nn.Conv2d(out_c, out_c, kernel_size = kernel_size, padding = padding),
            nn.BatchNorm2d(out_c),
            nn.ReLU(inplace = True))
        return conv
    
    def forward(self, x0):
        x1 = self.down_conv1(x0)
        p1 = self.max_pool2d(x1)
        x2 = self.down_conv2(p1)
        p2 = self.max_pool2d(x2)
        x3 = self.down_conv3(p2)
        p3 = self.max_pool2d(x3)
        x4 = self.down_conv4(p3)
        p4 = self.max_pool2d(x4)
        p4 = p4.view(-1, 28*28*128)
        x5 = self.linear1(p4)
        x5 = F.relu(x5)
        x6 = self.linear2(x5)
        x6 = F.relu(x6)
        x7 = self.linear3(x6)
        x7 = torch.softmax(x7, 1)
        
        return x7

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from clsfr_data_loading import lungCT_clsfr_DATA
import numpy as np
from tqdm as tqdm
import time
import os

pth = '/home/rrsood003/DATA/classifier_pairs.npy'
classifier_pairs = np.load(pth, allow_pickle = True)

batch_size = 16
train = lungCT_clsfr_DATA(classifier_pairs, 'train')
train_loader = DataLoader(train, batch_size = batch_size, shuffle = True)
val = lungCT_clsfr_DATA(classifier_pairs, 'val')
val_loader = DataLoader(val, batch_size = batch_size, shuffle = True)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
to_save = True
epochs = 10
lr = 0.01
model = lungPartitionClsfr(1, 3).to(device)
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = lr)

train_losses = []
val_losses = []
train_accuracies = []
val_accuracies = []
for epoch in range(1, epochs + 1):
    model.train()
    train_loss = 0
    val_loss = 0
    train_total = 0
    train_correct = 0
    val_total = 0
    val_correct = 0
    for batch in tqdm(train_loader):
        X_batch, y_batch = batch[0].to(device), batch[1].to(device)
        output_train = model(X_batch)
        loss_train = criterion(output_train, y_batch)
        loss_train.backward()
        optimizer.step()
        train_loss += loss_train.item()
        
        for idx, y in enumerate(y_batch):
            if torch.argmax(y) == torch.argmax(output_train[idx]):
                train_correct += 1
            train_total += 1
    
    with torch.no_grad():
        model.eval()        
        for batch_val in tqdm(val_loader):
            X_val, y_val = batch_val[0].to(device), batch_val[1].to(device)
            output_val = model(X_val)
            loss_val = criterion(output_val, y_val)
            val_loss += loss_val.item()

            for idx, y in enumerate(y_val):
                if torch.argmax(y) == torch.argmax(output_val[idx]):
                    val_correct += 1
                val_total += 1
    
    train_loss /= len(train_loader)
    val_loss /= len(val_loader)
    train_acc = train_correct / train_total
    val_acc = val_correct / val_total
    
    train_losses.append(train_loss)
    val_losses.append(val_loss)
    train_accuracies.append(train_acc)
    val_accuracies.append(val_acc)
    
    print("Epoch [%s]/[%s]\tTrain Loss: %s\tVal Loss: %s\tTrain Acc: %s\tVal Acc: %s" 
          % (epoch, epochs, train_loss, val_loss, train_acc, val_acc))

if to_save:
    date = time.strftime("%D").replace('/', '-')
    clsfr_file = 'lungCT_clsfr_epoch%s_batch%s_%s' % (epochs, batch_size, date)
    save_folder = '/home/rrsood003/Classifier/saved_models/' + clsfr_file
    os.mkdir(save_folder)
    torch.save(model, save_folder + clsfr_file + '.pth')
    np.save(save_folder + 'train_losses.npy', np.array(train_losses))
    np.save(save_folder + 'val_losses.npy', np.array(val_losses))
    np.save(save_folder + 'train_accuracies.npy', np.array(train_accuracies))
    np.save(save_folder + 'val_accuracies.npy', np.array(val_accuracies))
    
    
    
            